# DLC annotator tutorial
Please check this [example](https://github.com/DeepLabCut/DeepLabCut/blob/master/examples/JUPYTER/Demo_yourowndata.ipynb) for more info on this.
The DLC workshop on [GitHub](https://github.com/DeepLabCut/DeepLabCut-Workshop-Materials/blob/master/DLCcourse.md#the-basics-of-computing-in-python-terminal-and-overview-of-dlc) may also provide all the information you may require.
This is a notebook intended for my personal use, as I am getting to know and experiment with DLC.

## Import modules:

In [1]:
%load_ext autoreload
%autoreload 2

import deeplabcut
import os
from dlc_utils import *

## Create a project and configuration path:

In [2]:
mice_id = find_videos(r'C:\Users\jhflc\Documents\Projects\KI_article', extension='.avi')
print(mice_id)
vid_paths = get_paths(['blackfly_375529_2021-07-25_7_VIDEO.avi'],r'C:\Users\jhflc\Documents\Projects\KI_article\data')
print(vid_paths)

{'375529': 'blackfly_375529_2021-07-25_7_VIDEO.avi'}
['C:\\Users\\jhflc\\Documents\\Projects\\KI_article\\data\\375529_7\\blackfly_375529_2021-07-25_7_VIDEO.avi']


In [3]:
task='POSE' # Enter the name of your experiment Task
experimenter='JoaoCampagnolo' # Enter the name of the experimenter
video=['C:\\Users\\jhflc\\Documents\\Projects\\KI_article\\data\\375529_7\\blackfly_375529_2021-07-25_7_VIDEO.avi'] # Enter the paths of your videos OR FOLDER you want to grab frames from.

path_config_file=deeplabcut.create_new_project(task,experimenter,video,copy_videos=True)
# NOTE: The function returns the path, where your project is.

Created "C:\Users\jhflc\Documents\Projects\KI_article\scripts\POSE-JoaoCampagnolo-2021-11-24\videos"
Created "C:\Users\jhflc\Documents\Projects\KI_article\scripts\POSE-JoaoCampagnolo-2021-11-24\labeled-data"
Created "C:\Users\jhflc\Documents\Projects\KI_article\scripts\POSE-JoaoCampagnolo-2021-11-24\training-datasets"
Created "C:\Users\jhflc\Documents\Projects\KI_article\scripts\POSE-JoaoCampagnolo-2021-11-24\dlc-models"
Copying the videos
C:\Users\jhflc\Documents\Projects\KI_article\scripts\POSE-JoaoCampagnolo-2021-11-24\videos\blackfly_375529_2021-07-25_7_VIDEO.avi
Generated "C:\Users\jhflc\Documents\Projects\KI_article\scripts\POSE-JoaoCampagnolo-2021-11-24\config.yaml"

A new project with name POSE-JoaoCampagnolo-2021-11-24 is created at C:\Users\jhflc\Documents\Projects\KI_article\scripts and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have changed the configuration file, use the function 'extrac

### Or use pre-existing project

In [2]:
# Note that parameters of this project can be seen at: *Reaching-Mackenzie-2018-08-30/config.yaml*
from pathlib import Path

#create a variable to set the config.yaml file path:
path_config_file = os.path.join(os.getcwd(),
                                'C:\\Users\\jhflc\\Documents\\Projects\\KI_article\\scripts\\POSE-JoaoCampagnolo-2021-11-24\\config.yaml')
print(path_config_file)

C:\Users\jhflc\Documents\Projects\KI_article\scripts\POSE-JoaoCampagnolo-2021-11-24\config.yaml


## Extract fames from the videos

"A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) ('uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling."

In [4]:
%matplotlib inline
#there are other ways to grab frames, such as uniformly; please see the paper:

#AUTOMATIC:
deeplabcut.extract_frames(path_config_file) 

Config file read successfully.
Do you want to extract (perhaps additional) frames for video: C:\Users\jhflc\Documents\Projects\KI_article\scripts\POSE-JoaoCampagnolo-2021-11-24\videos\blackfly_375529_2021-07-25_7_VIDEO.avi ?
yes/noyes
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 1481.5  seconds.
Extracting and downsampling... 44445  frames from the video.


44445it [01:31, 487.77it/s]


Kmeans clustering ... (this might take a while)
Frames were successfully extracted, for the videos listed in the config.yaml file.

You can now label the frames using the function 'label_frames' (Note, you should label frames extracted from diverse videos (and many videos; we do not recommend training on single videos!)).


In [ ]:
#AND/OR:
#SELECT RARE EVENTS MANUALLY:
%gui wx
deeplabcut.extract_frames(path_config_file,'manual')

## Label the extracted frames

"Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling."

In [ ]:
%gui wx
deeplabcut.label_frames(path_config_file)

## Check the labels

"Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check_labels' to do so. It is used as follows:"

In [3]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by JoaoCampagnolo.


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 17.70it/s]

If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


## Create training dataset

"This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typcailly, 1 is what you will set, so you pass nothing!).

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!"

In [4]:
deeplabcut.create_training_dataset(path_config_file)
#remember, there are several networks you can pick, the default is resnet-50! Other types: resnet_101, resnet_152, 
#mobilenet_v2_1.0, mobilenet_v2_0.75, mobilenet_v2_0.5, mobilenet_v2_0.35, efficientnet-b0, efficientnet-b1, 
#efficientnet-b2, efficientnet-b3, efficientnet-b4, efficientnet-b5, and efficientnet-b6.

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([ 2, 17,  6, 10,  0,  8,  7, 15,  3, 16, 19, 11, 14,  5, 13, 18, 12,
           1,  4]),
   array([9])))]

## Training the model

"This function trains the network for a specific shuffle of the training dataset."

In [ ]:
deeplabcut.train_network(path_config_file)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['bodypart1', 'bodypart2', 'bodypart3', 'objectA'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'clahe': True,
 'claheratio': 0.1,
 'crop_pad': 0,
 'crop_sampling': 'hybrid',
 'crop_size': [400, 400],
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_POSENov24\\POSE_JoaoCampagnolo95shuffle1.mat',
 'dataset_type': 'default',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'edge': False,
 'emboss': {'alpha': [0.0, 1.0], 'embossratio': 0.1, 'strength': [0.5, 1.5]},
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'histeq': True,
 'histeqratio': 0.1,
 'init_weights': 'C:\\Users\\jhflc\\anaconda3\\envs\\DEEPLABCUT\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_we

Selecting single-animal trainer
Batch Size is 1


C:\Users\jhflc\anaconda3\envs\DEEPLABCUT\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Loading ImageNet-pretrained resnet_50
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'C:\\Users\\jhflc\\Documents\\Projects\\KI_article\\scripts\\POSE-JoaoCampagnolo-2021-11-24\\dlc-models\\iteration-0\\POSENov24-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'default', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 'all_joints': [[0], [1], [2], [3]], 'all_joints_names': ['bodypa

iteration: 1000 loss: 0.0300 lr: 0.005
iteration: 2000 loss: 0.0161 lr: 0.005
iteration: 3000 loss: 0.0122 lr: 0.005
iteration: 4000 loss: 0.0096 lr: 0.005
iteration: 5000 loss: 0.0090 lr: 0.005
iteration: 6000 loss: 0.0077 lr: 0.005
iteration: 7000 loss: 0.0075 lr: 0.005
iteration: 8000 loss: 0.0066 lr: 0.005
iteration: 9000 loss: 0.0065 lr: 0.005
iteration: 10000 loss: 0.0062 lr: 0.005
iteration: 11000 loss: 0.0092 lr: 0.02
iteration: 12000 loss: 0.0075 lr: 0.02
iteration: 13000 loss: 0.0067 lr: 0.02
iteration: 14000 loss: 0.0063 lr: 0.02
iteration: 15000 loss: 0.0056 lr: 0.02
iteration: 16000 loss: 0.0054 lr: 0.02
iteration: 17000 loss: 0.0054 lr: 0.02
iteration: 18000 loss: 0.0049 lr: 0.02
iteration: 19000 loss: 0.0049 lr: 0.02
iteration: 20000 loss: 0.0047 lr: 0.02
iteration: 21000 loss: 0.0045 lr: 0.02
iteration: 22000 loss: 0.0047 lr: 0.02
iteration: 23000 loss: 0.0044 lr: 0.02
iteration: 24000 loss: 0.0045 lr: 0.02
iteration: 25000 loss: 0.0043 lr: 0.02
iteration: 26000 loss: 0

## Evaluate the model

"This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .csv file in a subdirectory under **evaluation-results**"

In [ ]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

## Analyze the videos

"This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides."

In [ ]:
videofile_path = ['videos/video3.avi','videos/video4.avi'] #Enter a folder OR a list of videos to analyze.

deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.avi')

## Extract outlier frames [optional step]

"This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:"

In [ ]:
deeplabcut.extract_outlier_frames?

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi']) #pass a specific video

## Refine Labels [optional step]

"Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset."

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

"**NOTE**: Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: deeplabcut.label_frames(path_config_file)

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:
<code>deeplabcut.label_frames(path_config_file)</code>

Check Labels:

<code>deeplabcut.check_labels(path_config_file)</code>"

In [ ]:
#NOW, merge this with your original data:
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]

"Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!"

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video

"This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

THIS HAS MANY FUN OPTIONS!

<code>deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)</code>

So please check: <code>deeplabcut.create_labeled_video?</code> "

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos

"This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color."

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)